# Exercises 7 and 8 - Chapter 6

Train and fine-tune a Decision Tree for the moons dataset by following these steps: 
- Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset. 
- Use train_test_split() to split the dataset into a training set and a test set.
- Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.
- Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy.

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

In [4]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=10000, noise=0.4)

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Grid Search CV with DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state=1989)

from sklearn.model_selection import GridSearchCV

# Parameters to tune:
# max_leaf_nodes
# min_samples_split
# max_depth
# min_samples_leaf
max_leaf_nodes_params = list(range(2, 100, 5))
max_leaf_nodes_params.append(None)

param_grid_tree = {
    'max_leaf_nodes' : max_leaf_nodes_params,
    'min_samples_split': list(range(2, 4, 10)),
    'max_depth': [None, 1, 5, 10, 14, 20],
    'min_samples_leaf': [1, 3, 5, 10]   
}

In [17]:
grid = GridSearchCV(tree, param_grid_tree, cv = 3, scoring = 'accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=1989),
             param_grid={'max_depth': [None, 1, 5, 10, 14, 20],
                         'max_leaf_nodes': [2, 7, 12, 17, 22, 27, 32, 37, 42,
                                            47, 52, 57, 62, 67, 72, 77, 82, 87,
                                            92, 97, None],
                         'min_samples_leaf': [1, 3, 5, 10],
                         'min_samples_split': [2, 12, 22, 32]},
             scoring='accuracy')

In [18]:
grid.best_params_

{'max_depth': None,
 'max_leaf_nodes': 17,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [20]:
grid.best_score_

0.8555001986342105

In [21]:
# Now I should train the model on the whole training dataset
final_dt = DecisionTreeClassifier(random_state=42)
final_dt.set_params(**grid.best_params_)

final_dt.fit(X_train, y_train)

DecisionTreeClassifier(max_leaf_nodes=17, random_state=42)

In [23]:
from sklearn.metrics import accuracy_score
y_pred = final_dt.predict(X_test)
accuracy_score(y_test, y_pred)

0.8695

FUN FACT (from the By default, `GridSearchCV` trains the best model found on the whole training set (you can change this by setting `refit=False`)

8. Grow a forest by following these steps: 

- Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn’s ShuffleSplit class for this.
- Train one Decision Tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.
- Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This approach gives you majority-vote predictions over the test set. 
- Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!


In [ ]:
from sklearn.model_selection import ShuffleSplit

rs = ShuffleSplit(n_splits=1000, train_size=100, random_state=1989)

# The following line of code was unnecessary
rs.get_n_splits(X_train)

In [ ]:
# Iterate over the shuffledsplits
trees = []
for train_indexes, test_indexes in rs.split(X_train):
    new_tree = DecisionTreeClassifier(**grid.best_params_)
    new_tree.fit(X_train[train_indexes], y_train[train_indexes])
    trees.append(new_tree)

In [34]:
accuracy_scores = []
for tree in trees:
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

In [37]:
sum(accuracy_scores)/len(accuracy_scores)

0.8048190000000001

In [40]:
y_pred.shape

(2000,)

In [42]:
# For each test set instance, generate the predictions of the 1,000 decision trees and then take the mode() from SciPy
count = 0
for tree in trees: 
    y_pred = tree.predict(X_test)
    if count == 0:
        y_preds = y_pred
    else:
        y_preds = np.column_stack((y_preds, y_pred))
    count =+ 1

In [43]:
y_preds[:5]

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 1, ..., 1, 0, 1]])

In [44]:
from scipy.stats import mode
y_pred_mode = mode(y_preds, axis=1)

In [61]:
y_vote = y_pred_mode.mode.flatten()

In [58]:
isinstance(y_vote, np.ndarray)

True

In [62]:
accuracy_score(y_test, y_vote)

0.876

THIS WAS GOOD! But there are better ways, look at the solutions

and this: https://stackoverflow.com/questions/18691084/what-does-1-mean-in-numpy-reshape#comment87234675_42510505